In [1]:
#dependencias
import pandas as pd
import numpy as np
import requests as req
import json
import csv
import time
import threading as th

In [2]:
url1 = "http://bustime.noxxonsat.com.br/emtu"
url2 = "http://bustime.noxxonsat.com.br/bustime"
headers = {"Referer":"http://bustime.noxxonsat.com.br/emtu/emtu.html?linha=297"}
params = {"linha": "020"}

In [3]:
#carregar linhas
raw = pd.read_csv("emtu.tsv", delimiter="\t")
inds = (raw["LINHA"] != "LINHA") | pd.notnull(raw["LINHA"])
raw = raw[inds]
uinds = raw["INTEGRAÇÃO"] == "-"
dat = raw[uinds]
linhas = np.array(dat["LINHA"])

In [4]:
def reqThread(block, s):
    for linha in block:
        sl = time.time()
        params = {"linha": linha}        
        r2=s.get(url1, headers=headers, params=params)
        r3=s.get(url2, headers=headers, params=params)
        j = r3.json()
        if len(j['linhas']) > 0:
            for veiculo in j['linhas'][0]['veiculos']:
                dat.append([veiculo['codigoLinha'], veiculo['prefixo'], veiculo['latitude'],
                veiculo['longitude'], veiculo['sentidoLinha'], veiculo['dataUltimaTransmissao']])
    print((time.time() - start))

In [5]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [6]:
dat = []
s = req.session()

linhas_blocks = list(chunks(linhas, 100))

start = time.time()
end = 0
while 1 == 1:
    for block in linhas_blocks:
        th.Thread(target=reqThread, args=(block, s)).start()
        print("hi")
    delta = time.time() - start
    print("bu")
    if delta > 0:
        time.sleep(180 - delta)
    start = time.time()

hi
hi
hi
hi
hi
hi
bu
83.41999197006226


KeyboardInterrupt: 

In [8]:
with open("outp.csv", "w") as fil:
    csvw = csv.writer(fil)
    for d in dat:
        csvw.writerow(d)

123.21568894386292
124.30661630630493
127.55172419548035
129.2275414466858
131.4881477355957


In [7]:
len(dat)

2084